In [36]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain_core.prompts import ChatPromptTemplate
import cassio
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
apikey = os.getenv('GOOGLE_API_KEY')
ASTRA_DB_APPLICATION_TOKEN = os.getenv('ASTRA_DB_APPLICATION_TOKEN')
ASTRA_DB_ID = os.getenv('ASTRA_DB_ID')
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [26]:
loader = PyPDFLoader('DeathMystery.pdf')
text_doc=loader.load()

In [30]:
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs=text_splitter.split_documents(text_doc)

In [31]:
embeddings=OpenAIEmbeddings()
astra_vector_store=Cassandra(
    embedding=embeddings,
    table_name="mystery_with_ai",
    session=None,
    keyspace=None
)

In [32]:
astra_vector_store.add_documents(docs)
print("Inserted %i headlines." % len(docs))

astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 6 headlines.


In [37]:
# Retrieve Google API key from environment variables
apikey = os.getenv('GOOGLE_API_KEY')

# Configure Google Generative AI with the API key
genai.configure(api_key=apikey)

# Initialize the ChatGoogleGenerativeAI with the specified model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [39]:
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>

Question: {input}""")

In [41]:
print(astra_vector_index.query("tell me everything about mark",llm=llm))

Mark is David's brother who runs a small, semi-successful business. He always helped David with his business. While there were no apparent conflicts between the brothers, it was possible that Mark was jealous of David's rapid growth and success. In fact, Mark had secretly harbored resentment towards David for years, feeling that their parents always favored David. Recently, David had discovered that Mark had been embezzling money from the family business and was planning to confront him about it, putting Mark in a precarious position. 

The detective eventually arrested Mark for David's murder, deducing that David suspected his brother's jealousy and wrote the note to point authorities in his brother's direction.  

